In [1]:
import numpy as np
import pickle
import pandas as pd

In [2]:
class LiteOCR:
    def __init__(self,fn="alpha_weights.pkl",pool_size=2):
        [weights,meta]=pickle.load(open(fn,'rb'),encoding='latin1')
        
        self.vocab=meta['vocab']
        self.img_rows=meta["img_side"];self.img_cols=meta['img_side']
        
        #load CNN
        self.CNN=LiteCNN()
        #with saved weights
        self.CNN.load_weights(weights)
        
        self.CNN.pool_size=int(pool_size)
        
        
    def predict(self,image):
        print(image.shape)
        
        x=p.reshape(image,(1,1,self.img_rows,self.img_cols)).as_type('float32')
        
        #make prediction
        predicted_i=self.CNN.predict(x)
        
        return self.vocab(predicted_i)
    
class LiteCNN:
    def __init__(self):
        self.layers=[]
        self.pool_size=None
        
    def load_weights(self,weights):
        assert not self.layers,"weights can only be loaded once"
        
        for k in range(len(weights.keys())):
            self.layers.append(weights['layer_{}'.format(k)])
            
            
    def predict(self,X):
        #High Level Code
        h=self.cnn_layer(X,layer_i=0,border_mode="full");X=h
        h=self.relu_layer(X);X=h
        h=self.cnn_layer(X,layer_i=2,border_mode="valid");X=h
        h=self.relu_layer(X);X=h
        h=self.maxpooling_layer(X);X=h
        h=self.dropout_layer(X,.25);X=h
        h=self.flatten_layer(X,layer_i=7);X=h
        h=self.dense_layer(X,fully,layer_i=10);X=h
        h=self.softmax_layer2D(X);X=h
        max_i=self.classify(X)
        
        return max_i[0]
    
    #given our feature map we've learned from convolving around image
    
    def cnn_layer(self,X,layer_i=0,border_mode='full'):
        feature=self.layers[layers_i]["param_0"]
        bias=self.layers[layers_i]["param_1"]
        
        #how big is filter/patch?
        patch_dim=features[0].shape[-1]
        #how many features do we have?
        nb_features=features.shape[0]
        
        image_dim=X.shape[2]
        image_channel=X.shape[1]
        nb_images=X.shape[0]
        
        
        if border_mode == "full":
            conv_dim = image_dim + patch_dim - 1
        #With border mode "valid" you get an output that is smaller than the input because 
        #the convolution is only computed where the input and the filter fully overlap.
        elif border_mode == "valid":
            conv_dim = image_dim - patch_dim + 1
        
        convolved_features = np.zeros((nb_images, nb_features, conv_dim, conv_dim));
        for image_i in range(nb_images):

            for feature_i in range(nb_features):
                #lets initialize a convolved image as empty
                convolved_image = np.zeros((conv_dim, conv_dim))
                for channel in range(image_channels):
                    feature = features[feature_i, channel, :, :]
                    image   = X[image_i, channel, :, :]
                    convolved_image += self.convolve2d(image, feature, border_mode);
                convolved_image = convolved_image + bias[feature_i]
                convolved_features[image_i, feature_i, :, :] = convolved_image
        return convolved_features


    def dense_layer(self, X, layer_i=0):
        #so we'll initialize our weight and bias for this layer
        W = self.layers[layer_i]["param_0"]
        b = self.layers[layer_i]["param_1"]
        output = np.dot(X, W) + b
        return output

    def convolve2d(image, feature, border_mode="full"):

        image_dim = np.array(image.shape)
        feature_dim = np.array(feature.shape)
        #as well as a target dimension
        target_dim = image_dim + feature_dim - 1
        fft_result = np.fft.fft2(image, target_dim) * np.fft.fft2(feature, target_dim) 
        target = np.fft.ifft2(fft_result).real

        if border_mode == "valid":
            # To compute a valid shape, either np.all(x_shape >= y_shape) or
            valid_dim = image_dim - feature_dim + 1
            if np.any(valid_dim < 1):
                valid_dim = feature_dim - image_dim + 1
            start_i = (target_dim - valid_dim) // 2
            end_i = start_i + valid_dim
            target = target[start_i[0]:end_i[0], start_i[1]:end_i[1]]
        return target

    def relu_layer(x):
        #turn all negative values in a matrix into zeros
        z = np.zeros_like(x)
        return np.where(x>z,x,z)

    def softmax_layer2D(w):
        maxes = np.amax(w, axis=1)
        maxes = maxes.reshape(maxes.shape[0], 1)
        e = np.exp(w - maxes)
        dist = e / np.sum(e, axis=1, keepdims=True)
        return dist

    #affect the probability a node will be turned off by multiplying it
    #by a p values (.25 we define)
    def dropout_layer(X, p):
        retain_prob = 1. - p
        X *= retain_prob
        return X

    #get the largest probabililty value from the list
    def classify(X):
        return X.argmax(axis=-1)

    #tensor transformation, less dimensions
    def flatten_layer(X):
        flatX = np.zeros((X.shape[0],np.prod(X.shape[1:])))
        for i in range(X.shape[0]):
            flatX[i,:] = X[i].flatten(order='C')
        return flatX
    